# pdf Ingestion

In [1]:
#chroma db settings
import os
from chromadb.config import Settings
import chromadb

#define the chroma settings 
CHROMA_SETTINGS = chromadb.PersistentClient(path="db",
                                            settings=Settings(anonymized_telemetry=False))


In [1]:
#using langchain for heavy lifting of file
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

import os
# from constants import CHROMA_SETTINGS


persist_directory = "db"


def main():
    for root,dirs,files in os.walk("docs"):
        for file in files:
            if file.endswith(".pdf"):
                print(file)
                
                loader = PDFMinerLoader(os.path.join(root,file))

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                                    chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    #creating embeddings for storing 
    embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2"
                                               )
    #creating vectore store
    # vector_store = Chroma(embeddings,persist_directory=persist_directory)
    # vector_store.save()

    db = Chroma.from_documents(texts,
                               embeddings,persist_directory=persist_directory,)
                            #    client_settings=CHROMA_SETTINGS)
    db.persist()
    print("Successfull")
    db=None

if __name__ == "__main__":
    main()

the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf


KeyboardInterrupt: 

In [ ]:
# import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from constants import CHROMA_SETTINGS

checkpoint = 'LaMini-T5-738M'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,device_map="auto",
                                              torch_dtype=torch.float32,offload_folder='offload_weights')

# @st.cache_resource
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length = 256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95

    )

    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

# @st.cache_resource
def qa_llm():
    llm = llm_pipeline()
    embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")
    db = Chroma(persist_directory="db",
                embedding_function=embeddings)
                #client_settings=CHROMA_SETTINGS)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        retriever=retriever,
        # return_source_documnet = True
    )
    return qa

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    return answer, generated_text


# from Scratch

In [53]:
#loading Document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/ACEStar Awards Framework - FY24 v1.3.pdf")
pages = loader.load()

In [54]:
len(pages)

19

In [55]:
pages[5]

Document(page_content='©Larsen & Toubro Infotech Ltd. Privileged and Confidential\n•\n•\n•\n•\n•\n•\n•\n2', metadata={'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 5})

In [56]:
# document splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
# chunk_size =26
# chunk_overlap = 4
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [57]:

docs = text_splitter.split_documents(pages)
# docs = text_splitter.split_documents(sentence1)

In [58]:
len(docs)

19

In [61]:
print(pages[6].metadata,docs[3].metadata)

{'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 6} {'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 3}


In [62]:
#embeddings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.embeddings import SentenceTransformerEmbeddings

sentence1 = "i like dogs"
embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2"
                                               )


In [14]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

In [14]:
!rm -rf ./docs/chroma  # remove old database files if any

In [15]:
# vectordb = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     persist_directory=persist_directory
# )

In [63]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

# Important Faiss LInk

https://python.langchain.com/docs/integrations/vectorstores/faiss

In [64]:
query = "How many use cases were considered in the paper?"
docs = db.similarity_search(query,k=3)
docs

[Document(page_content='©Larsen & Toubro Infotech Ltd. Privileged and Confidential\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n* Team is considered as count 1•\n•\n•', metadata={'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 1}),
 Document(page_content='©Larsen & Toubro Infotech Ltd. Privileged and Confidential\n•\n•\n•\n•\n••\n•\n•\n•\n•To be \nfinalized', metadata={'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 15}),
 Document(page_content='©Larsen & Toubro Infotech Ltd. Privileged and Confidential\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n\uf076\n1\n2\n5\n3\n4', metadata={'source': 'docs/ACEStar Awards Framework - FY24 v1.3.pdf', 'page': 2})]

In [65]:
print(docs[0].page_content)

©Larsen & Toubro Infotech Ltd. Privileged and Confidential
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
* Team is considered as count 1•
•
•


In [20]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores[0]

(Document(page_content='Use cases  are targeted applications to a specific business challenge that produces one \nor more measurable outcomes. For example, in marketing, generative AI could be used to \ngenerate creative content such as personalized emails.\nUnstructured data  lack a consistent format or structure (for example, text, images, and audio \nfiles) and typically require more advanced techniques to extract insights.\n7 The economic potential of generative AI: The next productivity frontier', metadata={'source': 'docs/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf', 'page': 8}),
 1.0668528)

In [14]:
ans =db.similarity_search(query,k=5)

In [16]:
for doc in ans:
    print(doc.metadata,doc)

{'source': 'docs/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf', 'page': 8} page_content='Use cases  are targeted applications to a specific business challenge that produces one \nor more measurable outcomes. For example, in marketing, generative AI could be used to \ngenerate creative content such as personalized emails.\nUnstructured data  lack a consistent format or structure (for example, text, images, and audio \nfiles) and typically require more advanced techniques to extract insights.\n7 The economic potential of generative AI: The next productivity frontier' metadata={'source': 'docs/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf', 'page': 8}
{'source': 'docs/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf', 'page': 55} page_content='Our analysis of use cases only examines the direct impact of generative AI on productivity. \nThey do not incorporate secondary benefits, such as the econo

# Completed Project Below FOrmat 2

In [13]:
#loading Document
from langchain.document_loaders import PyPDFLoader
# text_loader_kwargs={'autodetect_encoding': True}
loader = PyPDFLoader("docs/43905.pdf")
pages = loader.load()
# document splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
# chunk_size =26
# chunk_overlap = 4
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

docs = text_splitter.split_documents(pages)

In [14]:
#embeddings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForCausalLM
from langchain.embeddings import SentenceTransformerEmbeddings

sentence1 = "i like dogs"
embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2"
                                               )



from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [15]:
#storing db and retriving 
db.save_local("faiss_index_test")
new_db = FAISS.load_local("faiss_index_test", embeddings)

In [4]:

#test
# import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForCausalLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
# from constants import CHROMA_SETTIN GS

# checkpoint = 'LaMini-T5-738M'
checkpoint = 'LaMini-Neo-1.3B'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForCausalLM.from_pretrained(checkpoint,device_map="auto",
                                              torch_dtype=torch.float32,offload_folder='offload_weights')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:

# @st.cache_resource
def llm_pipeline():
    pipe = pipeline(
        # 'text2text-generation',
        'text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length = 512,
        do_sample=True,
        temperature=1.5,
        top_p=0.90,
        eos_token_id=tokenizer.eos_token_id,
        # repetition_penalty=1.2

    )

    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

# @st.cache_resource
def qa_llm():
    llm = llm_pipeline()
    embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")
    retriever = new_db.as_retriever(search_type='similarity',k=4)
    
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff", #map-reduce,Refine,map-rerank
        retriever=retriever,
        return_source_documents = True
    )
    return qa#,retriever.metadata

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    return answer, generated_text#,metadata


In [24]:
query = "What is LSTM?"
output,x = process_answer(query)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
x

{'query': 'What is LSTM?',
 'result': ' A specifically designed recurrent neural network (RNN) architecture, specifically one designed for modelling input/cell input and its output, recurrent output, timestamp-ordered outputs, and cell-specific recurrent blocks. LSTM stands for Long Short-Term Memory and is often used for time-series and pattern recognition tasks.',
 'source_documents': [Document(page_content='Google, USA\n{hasim,andrewsenior,fsb@google.com }\nAbstract\nLong Short-Term Memory (LSTM) is a speciﬁc recurrent neu-\nral network (RNN) architecture that was designed to model tem-', metadata={'source': 'docs/43905.pdf', 'page': 0}),
  Document(page_content='inputg cell hitft\nctot\nrecurrent\noutputxtmt\nrtrt−1\nyt\nLSTM memory blocks\nFigure 1: LSTMP RNN architecture. A single memory block is\nshown for clarity.', metadata={'source': 'docs/43905.pdf', 'page': 1}),
  Document(page_content='An LSTM network computes a mapping from an input\nsequencex= (x1,...,x T)to an output se

In [27]:
print(x['source_documents'][0].page_content)
print(x['source_documents'][0].metadata['source'])
print(x['source_documents'][0].metadata['page'])

Generative AI is likely to 
have the biggest impact on knowledge work, particularly activities involving decision making and collaboration, which previously had the lowest potential for automation.
39 The economic potential of generative AI: The next productivity frontier
docs/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf
40


In [40]:
output

" Generative AI is likely to have the biggest impact on knowledge work, particularly activities involving decision making and collaboration, which previously had the lowest potential for automation, through the technology's potential to automate tasks that previously required human intelligence, including activities such as data analysis, software development, knowledge work creation, data visualization, and predictive modeling. The increased adoption of technology may accelerate the adoption of new technologies in general, including management, talent acquisition, and expert services. Furthermore, as a result of generative AI's potential, knowledge workers may not have to manually program data analysis and decision-making software to complete their work."

In [12]:
print('Query : ',output[1]['query'],'\n')

print('Answer : ',output[1]['result'],'\n')
for x in output[1]['source_documents']:
    print('Source :',x,'\n')

TypeError: string indices must be integers

# above code modification for chat option by including memory option (Buffer Memory)

In [59]:
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory,ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=5,
)
# pipe = pipeline(
#     # 'text2text-generation',
#     'text-generation',
#     model=base_model,
#     tokenizer=tokenizer,
#     max_length = 512,
#     do_sample=True,
#     temperature=1.5,
#     top_p=0.90,
#     # repetition_penalty=1.2

# )

# local_llm = HuggingFacePipeline(pipeline=pipe)

# memory = ConversationSummaryMemory(llm=local_llm,chat_memory='chat_history',return_messages=True)
# memory.save_context({"input": "hi"}, {"output": "whats up"})

In [53]:
memory.load_memory_variables({})

{'history': '\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF NEW COUPPOS'}

In [68]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
# @st.cache_resource
chat_history=[]
def llm_pipeline():
    pipe = pipeline(
        # 'text2text-generation',
        'text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length = 512,
        do_sample=True,
        temperature=1,
        top_p=0.90

    )

    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

# @st.cache_resource
def qa_llm():
    llm = llm_pipeline()
    retriever = db.as_retriever()   
    qa = ConversationalRetrievalChain.from_llm(
        llm = llm,
        chain_type= "stuff", #map-reduce,Refine,map-rerank
        retriever=retriever,
        memory = memory,
        # return_source_documents = True,
        # return_generated_question=True
    )
    return qa#,retriever.metadata

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa({"question":instruction,"chat_history":chat_history})
    chat_history.extend([(query, generated_text["answer"])])
    # print(generated_text)
    answer = generated_text['answer']
    print(chat_history)
    return answer#, generated_text#,metadata


In [69]:
query = "how much monetory amount is affected by generative ai in future?"
output = process_answer(query)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [62]:
query = "do you have any other answer from above?"
output = process_answer(query)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('how much monetory amount is affected by generative ai in future?', " Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries.\nAnswer: Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries. \n\nTherefore, it is safe to assume that generative AI's monetiary potential will likely be affected by increased investment and usage in the future."), ('do you have any other answer from above?', ' Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries.\nThe economic potential of generative AI: The next productivity frontier')]


' Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries.\nThe economic potential of generative AI: The next productivity frontier'

In [65]:
query = "what if gdp is not affected by generative ai?"
output = process_answer(query)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('how much monetory amount is affected by generative ai in future?', " Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries.\nAnswer: Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries. \n\nTherefore, it is safe to assume that generative AI's monetiary potential will likely be affected by increased investment and usage in the future."), ('do you have any other answer from above?', ' Generative AI has the potential to generate $2.6 trillion to $4.4 trillion in value across industries.\nThe economic potential of generative AI: The next productivity frontier'), ('what all are the major areas in which these cases fall or have major impact on?', ' The market demand for generative AI technology can be influenced by several factors such as market size, consumer demand, technological advancements, regulatory environment, and business needs. \nTo determine the potential monetiary impact of gener

' It is unclear whether Generative AI will lead to more jobs and increase GDP based on the context given.'

# Conversation Summary Memory